<a href="https://colab.research.google.com/github/springboardmentor5432x/DV---Optimizing-IT-Support-Team-Performance-Using-Analytics-Supportlytics-/blob/Gokul-Optimizing-IT-Support-Team-Performance-Using-Analytics-(-Supportlytics)/milestone_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Milestone 1: Week 1-2**

 Module 1:- Project Initialization and Dataset Setup

In [11]:
import pandas as pd
import numpy as np

In [12]:
from google.colab import files
uploaded = files.upload()
df = pd.read_csv('Supportlytics.csv')
df.head()

,Ticket_ID,Status,Priority,Source,Topic,Category,Agent_Group,Created_time,Expected_SLA_resolve,Resolution_Duration,SLA_Status,Survey_Results,Country,Latitude,Longitude,Agent_interactions
0,TIC-5000,Closed,Critical,Chat,Access Revoked,Security,Network Ops,25-11-2025 07:14,4,2.84,On-Time,3.0,Germany,51.16,10.45,6
1,TIC-5001,Closed,Medium,Email,Wi-Fi Access,Network,Infrastructure,19-07-2025 11:09,24,27.19,Breached,4.0,Germany,51.16,10.45,8
2,TIC-5002,Resolved,Critical,Chat,Printer,Hardware,L2 Technical Support,01-05-2025 22:29,4,2.96,On-Time,NaN,India,20.59,78.96,2
3,TIC-5003,Closed,High,Email,Bug Report,Software,Security Admin,19-06-2025 07:21,8,1.89,On-Time,2.0,UK,55.37,-3.43,8
4,TIC-5004,Closed,Low,Chat,Update Failure,Software,L1 Service Desk,17-01-2025 06:30,48,10.85,On-Time,2.0,Brazil,-14.23,-51.92,3


In [13]:
df['Created_time'] = pd.to_datetime(df['Created_time'], dayfirst=True)

class DataProfiler:
    def __init__(self, data):
        self.data = data

    def health_check(self):
        profile = pd.DataFrame({
            'Data Type': self.data.dtypes,
            'Missing Values': self.data.isnull().sum(),
            'Unique Values': self.data.nunique(),
            'Completeness (%)': (1 - self.data.isnull().sum() / len(self.data)) * 100
        })
        return profile

# Execute Health Check
profiler = DataProfiler(df)
print("\n--- Advanced Data Health Report ---")
display(profiler.health_check())


--- Advanced Data Health Report ---


,Data Type,Missing Values,Unique Values,Completeness (%)
Ticket_ID,object,0,5000,100.00
Status,object,0,3,100.00
Priority,object,0,4,100.00
Source,object,0,4,100.00
Topic,object,0,16,100.00
Category,object,0,4,100.00
Agent_Group,object,0,5,100.00
Created_time,datetime64[ns],0,4974,100.00
Expected_SLA_resolve,int64,0,4,100.00
Resolution_Duration,float64,236,3109,95.28


In [14]:
# 1. Flagging original missing values (Requirement for Milestone 1)
df['Resolution_Imputed'] = df['Resolution_Duration'].isnull().astype(int)
df['Survey_Imputed'] = df['Survey_Results'].isnull().astype(int)

# 2. Advanced Imputation for Resolution Duration
# Only impute for Closed/Resolved tickets to avoid skewing 'Open' ticket metrics
mask = (df['Status'].isin(['Closed', 'Resolved'])) & (df['Resolution_Duration'].isnull())
df.loc[mask, 'Resolution_Duration'] = df[mask].groupby(['Category', 'Priority'])['Resolution_Duration'].transform(
    lambda x: x.fillna(x.median() if not np.isnan(x.median()) else df['Resolution_Duration'].median())
)

# 3. Neutral Imputation for Survey Results
avg_sat = df['Survey_Results'].mean()
df['Survey_Results'] = df['Survey_Results'].fillna(avg_sat)

print("\n[Data Cleaning Complete]: Missing values handled using Grouped Median and Global Mean logic.")

# Metric Definitions & Distributions

# Metric A: Resolution Gap (Expected vs Actual)
df['Resolution_Gap'] = df['Expected_SLA_resolve'] - df['Resolution_Duration']

# Metric B: Response Efficiency (Expected SLA / Interactions)
df['Response_Efficiency'] = df['Expected_SLA_resolve'] / (df['Agent_interactions'] + 1)

# Metric C: Initial Distribution Analysis
print("\n--- Ticket Distribution: Priority ---")
display(df['Priority'].value_counts().to_frame())

print("\n--- Multi-Factor Distribution: Category x Priority ---")
dist_matrix = pd.crosstab(df['Category'], df['Priority'], margins=True, margins_name="Total")
display(dist_matrix)


[Data Cleaning Complete]: Missing values handled using Grouped Median and Global Mean logic.

--- Ticket Distribution: Priority ---


,count
Priority,
Medium,2003
Low,1528
High,975
Critical,494



--- Multi-Factor Distribution: Category x Priority ---


Priority,Critical,High,Low,Medium,Total
Category,,,,,
Hardware,116,255,412,515,1298
Network,126,236,406,487,1255
Security,121,246,343,485,1195
Software,131,238,367,516,1252
Total,494,975,1528,2003,5000


**Module** 2: Similarity Level & Performance Bucketing

In [15]:


# 1. Normalize the metrics to a 0-1 scale for comparison
# Speed Score: 1 is best (instant), 0 is worst (slow)
df['Normalized_Speed'] = 1 - (df['Resolution_Duration'] / df['Expected_SLA_resolve'].replace(0, 1))
# Satisfaction Score: 1 is best (5/5), 0 is worst (1/5)
df['Normalized_Satisfaction'] = (df['Survey_Results'] - 1) / 4

# 2. Define the Similarity_Level (Weighted Average)
# We give 60% weight to SLA Speed and 40% to Customer Satisfaction
df['Similarity_Score'] = (df['Normalized_Speed'] * 0.6) + (df['Normalized_Satisfaction'] * 0.4)

# Handle cases where Speed might be negative (massive SLA breach)
df['Similarity_Score'] = df['Similarity_Score'].clip(lower=0, upper=1)

print("Similarity Scores calculated based on Speed and Satisfaction.")

Similarity Scores calculated based on Speed and Satisfaction.


*  Categorizing Similarity_Level into performance buckets

In [16]:

def get_performance_bucket(score):
    if pd.isna(score): return "Pending"
    if score >= 0.75: return "High Performer"
    if score >= 0.40: return "Standard"
    return "Low Efficiency"

df['Performance_Bucket'] = df['Similarity_Score'].apply(get_performance_bucket)

# 4. Final Review of the Buckets
bucket_dist = df['Performance_Bucket'].value_counts()
print("\n--- Performance Bucket Distribution ---")
print(bucket_dist)


--- Performance Bucket Distribution ---
Performance_Bucket
Low Efficiency    3022
Standard          1481
High Performer     261
Pending            236
Name: count, dtype: int64


In [18]:
df.to_csv('cleaned_Supportlytics.csv', index=False)